In [18]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [2]:
!pip install -q transformers datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.4 MB/s eta 0:00:00:00:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.

In [10]:
from datasets import load_dataset


dataset = load_dataset("baebee/Little-Literature")
print(dataset["train"][0])


{'text': 'In the heart of a bustling city, there stood a quaint bookstore, its windows adorned with the soft glow of a solitary lamp, illuminating the world within. It was a sanctuary for the lost, the dreamers, and the curious souls who sought solace in the comforting embrace of stories woven by the masters of words.'}


In [28]:
from transformers import AutoTokenizer, AutoModelForCausalLM


tokenizer = AutoTokenizer.from_pretrained("isarth/distill_gpt2_story_generator")
model = AutoModelForCausalLM.from_pretrained("isarth/distill_gpt2_story_generator")
tokenizer.pad_token = tokenizer.eos_token


In [33]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/4154 [00:00<?, ? examples/s]

In [34]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [35]:
print(tokenized_datasets["train"][0])


{'text': 'In the heart of a bustling city, there stood a quaint bookstore, its windows adorned with the soft glow of a solitary lamp, illuminating the world within. It was a sanctuary for the lost, the dreamers, and the curious souls who sought solace in the comforting embrace of stories woven by the masters of words.', 'input_ids': [818, 262, 2612, 286, 257, 46609, 1748, 11, 612, 6204, 257, 50127, 44346, 11, 663, 9168, 41860, 351, 262, 2705, 19634, 286, 257, 25565, 20450, 11, 46717, 262, 995, 1626, 13, 632, 373, 257, 24050, 329, 262, 2626, 11, 262, 4320, 364, 11, 290, 262, 11040, 15625, 508, 7194, 1540, 558, 287, 262, 34228, 12553, 286, 3923, 36932, 416, 262, 18159, 286, 2456, 13, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 50258, 5

In [36]:
from transformers import DataCollatorForLanguageModeling, TrainingArguments, Trainer, TrainerCallback

class LoggingCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs:
            print(f"Step: {state.global_step} - Loss: {logs.get('loss')}")
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
training_args = TrainingArguments(
    output_dir="./results",
    run_name="distilgpt2-finetune-run",
    logging_steps=10,        # Log every 10 steps
    save_steps=500,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,  # Accumulate gradients over 8 steps (2*8 = 16 samples per update)
    num_train_epochs=3,      # Train for 3 epochs
    weight_decay=0.01,
    save_total_limit=2,
    report_to=[]
)


In [37]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    data_collator=data_collator,
)



In [38]:
trainer.train()


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,2.214100
20,2.068600
30,2.237700
40,1.726600
50,1.856700
60,1.945500
70,1.600200
80,1.803800
90,1.571000
100,1.556900


TrainOutput(global_step=387, training_loss=1.4786362216762179, metrics={'train_runtime': 1283.1294, 'train_samples_per_second': 9.712, 'train_steps_per_second': 0.302, 'total_flos': 3240079707340800.0, 'train_loss': 1.4786362216762179, 'epoch': 2.9932627526467757})

In [42]:
from transformers import pipeline

text_gen = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer)

prompt = '''f"Write a vivid, emotionally grounded backstory for the following character. Avoid meta-commentary, real-world references, or speculation.\n Prompt: {"name": "Themis", "description": "Birth and early days: She was born from Gaia along with her younger sister Mnemosyne. She became the embodiment of divine order, law, and custom, giving the Titans a purpose to be part of the world. After Cronos overthrew and slayed his father Ouranos, she and several titans became his close allies. She pledged herself to him, but disagreed with his cruel methods. Powers & Abilities: Immortality — she is immortal and only a sufficiently powerful weapon or being can kill her. Divination — she can foresee the future and see the past of every being. Omniscience — she can see the past, present, and future of every creature."}
Backstory: '''

output = text_gen(prompt, max_new_tokens=200, repetition_penalty=1.2, do_sample=True, temperature=0.4)

print(output[0]['generated_text'])


Device set to use cuda:0


f"Write a vivid, emotionally grounded backstory for the following character. Avoid meta-commentary, real-world references, or speculation.
 Prompt: {"name": "Themis", "description": "Birth and early days: She was born from Gaia along with her younger sister Mnemosyne. She became the embodiment of divine order, law, and custom, giving the Titans a purpose to be part of the world. After Cronos overthrew and slayed his father Ouranos, she and several titans became his close allies. She pledged herself to him, but disagreed with his cruel methods. Powers & Abilities: Immortality — she is immortal and only a sufficiently powerful weapon or being can kill her. Divination — she can foresee the future and see the past of every being. Omniscience — she can see the past, present, and future of every creature."}
Backstory:  The story begins in this small town named after its ancient owner - an old man who had been watching through trees as if performing magic on their own deathbeds before he died

In [43]:
trainer.save_model("./backstory-model")
tokenizer.save_pretrained("./backstory-model")


('./backstory-model/tokenizer_config.json',
 './backstory-model/special_tokens_map.json',
 './backstory-model/vocab.json',
 './backstory-model/merges.txt',
 './backstory-model/added_tokens.json',
 './backstory-model/tokenizer.json')

In [44]:
import shutil
shutil.make_archive("backstory_model", 'zip', "/kaggle/working/backstory-model")  # ("output_name", format, "folder_path")

'/kaggle/working/backstory_model.zip'

In [45]:
from IPython.display import FileLink
FileLink('backstory_model.zip')  # click this link in notebook to download


/kaggle/working/backstory_model.zip